# Limpeza das bases

---

- Checagem dos IDs únicos
- Filtragem de alunos e escolas estaduais de ensino médio no RJ

Bases:
- Censo (2016 e 2017): dados de Matrículas e Escolas
- INSE (2015)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Censo-escolar" data-toc-modified-id="Censo-escolar-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Censo escolar</a></span><ul class="toc-item"><li><span><a href="#Matrículas" data-toc-modified-id="Matrículas-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Matrículas</a></span><ul class="toc-item"><li><span><a href="#Filtragem-de-alunos-em-escolas-estaduais-do-EM" data-toc-modified-id="Filtragem-de-alunos-em-escolas-estaduais-do-EM-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Filtragem de alunos em escolas estaduais do EM</a></span></li><li><span><a href="#Checando-ID-único" data-toc-modified-id="Checando-ID-único-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Checando ID único</a></span></li></ul></li><li><span><a href="#Escolas" data-toc-modified-id="Escolas-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Escolas</a></span><ul class="toc-item"><li><span><a href="#Filtrando-escolas-estaduais" data-toc-modified-id="Filtrando-escolas-estaduais-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Filtrando escolas estaduais</a></span></li></ul></li><li><span><a href="#INSE" data-toc-modified-id="INSE-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>INSE</a></span></li><li><span><a href="#Ideb" data-toc-modified-id="Ideb-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Ideb</a></span></li></ul></li><li><span><a href="#Fogo-Cruzado" data-toc-modified-id="Fogo-Cruzado-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Fogo Cruzado</a></span></li></ul></div>

In [3]:
from paths import RAW_PATH, TREAT_PATH
import os
import pandas as pd

## Censo escolar 

### Matrículas
- Filtragem de alunos do RJ no 1º ano do EM em escolas públicas estaduais (em 2016)
- Checagem do ID único: `ID_MATRICULA` não se mantém de um ano para outro (< 2% de 2016 aparece em 2017), usamos o `ID` do CPF criptografado
- Separando IDs duplicados

In [5]:
def unique_id(df):
    
    # Código único por pessoa
    v = df['ID'].value_counts()
    print('CPFs que aparecem mais de uma vez: ', len(v[v > 1]))

    # Separando duplicados
    duplicate = df[df['ID'].isin(v[v > 1].index)]
    df = df[df['ID'].isin(v[v > 1].index) == False]
    
    return df, duplicate

def clean_aluno(df_name):
    
    df = pd.read_csv(RAW_PATH / 'censo' / df_name)
    
    # Criptografa os CPFs
    df['ID'] = df['CO_PESSOA_FISICA']# .apply(hash)
    df = df.drop('CO_PESSOA_FISICA', axis=1)
    
    # Checa duplicados
    df, df_duplicate = unique_id(df)
    
    # Salva as bases tratadas
    clean_name = 'cleaned_'+str(df_name)
    duplicate_name = df_name[:-4]+'duplicado.csv'
    
    df.to_csv(TREAT_PATH / 'censo' / clean_name)
    df_duplicate.to_csv(TREAT_PATH / 'censo' / duplicate_name)
    
dfs = list(filter(lambda x: x.endswith('.csv'), os.listdir(RAW_PATH / 'censo')))

for df in dfs:
    if 'matricula' in df:
        clean_aluno(df)

CPFs que aparecem mais de uma vez:  331
CPFs que aparecem mais de uma vez:  687


In [7]:
df_aluno17 = pd.read_csv(TREAT_PATH / 'censo' / 'cleaned_matriculas_rj_2017.csv', index_col=0)
df_aluno16 = pd.read_csv(TREAT_PATH / 'censo' / 'cleaned_matriculas_rj_2016.csv', index_col=0)

In [8]:
df_aluno17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 551630 entries, 0 to 552291
Data columns (total 92 columns):
NU_ANO_CENSO                     551630 non-null int64
ID_MATRICULA                     551630 non-null int64
NU_DIA                           551630 non-null int64
NU_MES                           551630 non-null int64
NU_ANO                           551630 non-null int64
NU_IDADE_REFERENCIA              551630 non-null int64
NU_IDADE                         551630 non-null int64
NU_DURACAO_TURMA                 551630 non-null int64
NU_DUR_ATIV_COMP_MESMA_REDE      551630 non-null int64
NU_DUR_ATIV_COMP_OUTRAS_REDES    551630 non-null int64
NU_DUR_AEE_MESMA_REDE            551630 non-null int64
NU_DUR_AEE_OUTRAS_REDES          551630 non-null int64
NU_DIAS_ATIVIDADE                551630 non-null int64
TP_SEXO                          551630 non-null int64
TP_COR_RACA                      551630 non-null int64
TP_NACIONALIDADE                 551630 non-null int64
CO_PAIS_O

In [9]:
df_aluno16.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 570111 entries, 0 to 571484
Data columns (total 92 columns):
NU_ANO_CENSO                     570111 non-null int64
ID_MATRICULA                     570111 non-null int64
NU_DIA                           570111 non-null int64
NU_MES                           570111 non-null int64
NU_ANO                           570111 non-null int64
NU_IDADE_REFERENCIA              570111 non-null int64
NU_IDADE                         570111 non-null int64
NU_DURACAO_TURMA                 570111 non-null int64
NU_DUR_ATIV_COMP_MESMA_REDE      570111 non-null int64
NU_DUR_ATIV_COMP_OUTRAS_REDES    570111 non-null int64
NU_DUR_AEE_MESMA_REDE            570111 non-null int64
NU_DUR_AEE_OUTRAS_REDES          570111 non-null int64
NU_DIAS_ATIVIDADE                570111 non-null int64
TP_SEXO                          570111 non-null int64
TP_COR_RACA                      570111 non-null int64
TP_NACIONALIDADE                 570111 non-null int64
CO_PAIS_O

#### Filtragem de alunos em escolas estaduais do EM

In [10]:
df_aluno16 = df_aluno16[df_aluno16['TP_ETAPA_ENSINO'].isin([25, 31])] 
df_aluno16 = df_aluno16[df_aluno16['TP_DEPENDENCIA'] == 2] # Escolas estaduais

df_aluno16.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182838 entries, 118 to 553389
Data columns (total 92 columns):
NU_ANO_CENSO                     182838 non-null int64
ID_MATRICULA                     182838 non-null int64
NU_DIA                           182838 non-null int64
NU_MES                           182838 non-null int64
NU_ANO                           182838 non-null int64
NU_IDADE_REFERENCIA              182838 non-null int64
NU_IDADE                         182838 non-null int64
NU_DURACAO_TURMA                 182838 non-null int64
NU_DUR_ATIV_COMP_MESMA_REDE      182838 non-null int64
NU_DUR_ATIV_COMP_OUTRAS_REDES    182838 non-null int64
NU_DUR_AEE_MESMA_REDE            182838 non-null int64
NU_DUR_AEE_OUTRAS_REDES          182838 non-null int64
NU_DIAS_ATIVIDADE                182838 non-null int64
TP_SEXO                          182838 non-null int64
TP_COR_RACA                      182838 non-null int64
TP_NACIONALIDADE                 182838 non-null int64
CO_PAIS

#### Checando ID único

- Ambos são únicos por ano, mas `ID_MATRICULA` não se mantém de um ano para outro

In [11]:
df_aluno16['ID_MATRICULA'].value_counts()[:5]

208668671    1
210148729    1
210888086    1
211418517    1
211154324    1
Name: ID_MATRICULA, dtype: int64

In [12]:
df_aluno17['ID_MATRICULA'].value_counts()[:5]

222300159    1
247419713    1
237785889    1
247233316    1
247231269    1
Name: ID_MATRICULA, dtype: int64

- ID_MATRICULA: Só 1,4% de 2016 aparece em 2017

In [13]:
n = len(df_aluno16[df_aluno16['ID_MATRICULA'].isin(df_aluno17['ID_MATRICULA'])])

print('ID_MATRICULA de 2016 que aparece em 2017:', n)
print('Percentual (de 2016):', 100*n/len(df_aluno16))

ID_MATRICULA de 2016 que aparece em 2017: 2590
Percentual (de 2016): 1.4165545455539876


- ID (CPF criptografado): Mais de 60% de 2016 aparece em 2017 (mais razoável, dado que entram novos alunos e saem também)

In [14]:
n = len(df_aluno16[df_aluno16['ID'].isin(df_aluno17['ID'])])

print('ID de 2016 que aparece em 2017:', n)
print('Percentual (de 2016):', 100*n/len(df_aluno16))

ID de 2016 que aparece em 2017: 120955
Percentual (de 2016): 66.15419114188516


In [15]:
del df_aluno16['ID_MATRICULA']
del df_aluno17['ID_MATRICULA']

* Total de escolas dos alunos de 2016:

In [16]:
print('Total de escolas:', len(df_aluno16['CO_ENTIDADE'].unique()))

Total de escolas: 1097


### Escolas

- Total de 166 variáveis, 121 indicadores

In [19]:
escolas_16 = pd.read_csv(RAW_PATH / 'censo' / 'escolas_2016.csv', encoding='latin1', sep='|')
escolas_16.info()

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (3,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279358 entries, 0 to 279357
Columns: 166 entries, NU_ANO_CENSO to IN_ESP_EXCLUSIVA_PROF
dtypes: float64(151), int64(11), object(4)
memory usage: 353.8+ MB


In [20]:
print('Número de variáveis:', len(escolas_16.columns.values))

Número de variáveis: 166


In [21]:
indicadores = [i for i in escolas_16.columns if i.startswith('IN')]
print('Número de indicadores:', len(indicadores))

Número de indicadores: 121


#### Filtrando escolas estaduais

- 1689 escolas estaduais no RJ

In [22]:
escolas_16 = escolas_16[escolas_16['TP_DEPENDENCIA'] == 2] # Escolas estaduais
escolas_16 = escolas_16[escolas_16['CO_UF'] == 33] # no RJ

escolas_16.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1689 entries, 4012 to 275339
Columns: 166 entries, NU_ANO_CENSO to IN_ESP_EXCLUSIVA_PROF
dtypes: float64(151), int64(11), object(4)
memory usage: 2.2+ MB


In [23]:
escolas_16.to_csv(TREAT_PATH / 'censo' / 'cleaned_escolas_rj_2016.csv')

### INSE

- 936 escolas possuem nível socioeconômico

In [24]:
# Nível socioeconômico
# ! pip3 install xlrd

df_inse_2015 = pd.read_excel(RAW_PATH / 'INSE_2015.xlsx')
df_inse_2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63258 entries, 0 to 63257
Data columns (total 12 columns):
CO_ESCOLA              63258 non-null int64
NOME_ESCOLA            63258 non-null object
CO_UF                  63258 non-null int64
NOME_UF                63258 non-null object
CO_MUNICIPIO           63258 non-null int64
NOME_MUNICIPIO         63258 non-null int64
ID_AREA                63258 non-null int64
TP_DEPENDENCIA         63258 non-null int64
TP_LOCALIZACAO         63258 non-null int64
QTD_ALUNOS_INSE        63258 non-null int64
INSE_VALOR_ABSOLUTO    63258 non-null float64
INSE_CLASSIFICACAO     63258 non-null object
dtypes: float64(1), int64(8), object(3)
memory usage: 5.8+ MB


In [25]:
df_inse_2015 = df_inse_2015[df_inse_2015['TP_DEPENDENCIA'] == 2] # Escolas estaduais
df_inse_2015 = df_inse_2015[df_inse_2015['CO_UF'] == 33] # no RJ

In [26]:
df_inse_2015['NIVEL'] = df_inse_2015['INSE_CLASSIFICACAO'].apply(lambda x: int(x.split()[1]))
df_inse_2015.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 936 entries, 33485 to 37804
Data columns (total 13 columns):
CO_ESCOLA              936 non-null int64
NOME_ESCOLA            936 non-null object
CO_UF                  936 non-null int64
NOME_UF                936 non-null object
CO_MUNICIPIO           936 non-null int64
NOME_MUNICIPIO         936 non-null int64
ID_AREA                936 non-null int64
TP_DEPENDENCIA         936 non-null int64
TP_LOCALIZACAO         936 non-null int64
QTD_ALUNOS_INSE        936 non-null int64
INSE_VALOR_ABSOLUTO    936 non-null float64
INSE_CLASSIFICACAO     936 non-null object
NIVEL                  936 non-null int64
dtypes: float64(1), int64(9), object(3)
memory usage: 102.4+ KB


In [27]:
df_inse_2015.to_csv(TREAT_PATH / 'inse_2015_rj.csv')

### Ideb

In [17]:
df_ideb = pd.read_excel(RAW_PATH / 'saeb' / 'divulgacao_anos_finais-escolas-2017.xlsx', header=4)
df_ideb.head()

,Sigla da UF,Código do Município,Nome do Município,Código da Escola,Nome da Escola,Rede,Taxa de Aprovação - 2005,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,IDEB 2015 (N x P),IDEB 2017 (N x P),Projeções,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,6º a 9º ano,6º,7º,8º,...,NaN,NaN,2007,2009,2011,2013,2015,2017,2019,2021
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RO,1100015.0,Alta Floresta D'Oeste,11024666.0,EMEIEF BOA ESPERANCA,Municipal,-,-,-,-,...,-,4.1,-,-,4.3,4.6,4.9,5.2,5.4,5.7
4,RO,1100015.0,Alta Floresta D'Oeste,11024682.0,EEEF EURIDICE LOPES PEDROSO,Estadual,81,75.6,86.2,84.5,...,-,-,3.9,4.1,4.3,4.7,5.1,5.4,5.6,5.8


In [18]:
cols = df_ideb.columns

new_cols_name = ['sigla_uf', 'cod_municipio', 'nome_municipio', 'cod_inep', 'nome_escola', 'rede']
anos = [2005 + 2*i for i in range(7)]

taxa = ['taxa_aprovacao_{}_{}'.format(ano, serie) for ano in anos for serie in ['todos']+list(range(6,10))+['indicador']]
saeb = ['saeb_{}_{}'.format(ano, mod) for ano in anos for mod in ['mat', 'pt', 'media_padrao']]
ideb = ['ideb_{}'.format(ano) for ano in anos]
projecoes = ['projecoes_{}'.format(ano) for ano in anos[1:]+['2019', '2021']]


new_cols_name  = new_cols_name + taxa + saeb + ideb + projecoes

ideb_dic_cols = dict(zip(new_cols_name, cols))
ideb_dic_cols

{'sigla_uf': 'Sigla da UF',
 'cod_municipio': 'Código do Município',
 'nome_municipio': 'Nome do Município',
 'cod_inep': 'Código da Escola',
 'nome_escola': 'Nome da Escola',
 'rede': 'Rede',
 'taxa_aprovacao_2005_todos': 'Taxa de Aprovação - 2005',
 'taxa_aprovacao_2005_6': 'Unnamed: 7',
 'taxa_aprovacao_2005_7': 'Unnamed: 8',
 'taxa_aprovacao_2005_8': 'Unnamed: 9',
 'taxa_aprovacao_2005_9': 'Unnamed: 10',
 'taxa_aprovacao_2005_indicador': 'Unnamed: 11',
 'taxa_aprovacao_2007_todos': 'Taxa de Aprovação - 2007',
 'taxa_aprovacao_2007_6': 'Unnamed: 13',
 'taxa_aprovacao_2007_7': 'Unnamed: 14',
 'taxa_aprovacao_2007_8': 'Unnamed: 15',
 'taxa_aprovacao_2007_9': 'Unnamed: 16',
 'taxa_aprovacao_2007_indicador': 'Unnamed: 17',
 'taxa_aprovacao_2009_todos': 'Taxa de Aprovação - 2009',
 'taxa_aprovacao_2009_6': 'Unnamed: 19',
 'taxa_aprovacao_2009_7': 'Unnamed: 20',
 'taxa_aprovacao_2009_8': 'Unnamed: 21',
 'taxa_aprovacao_2009_9': 'Unnamed: 22',
 'taxa_aprovacao_2009_indicador': 'Unnamed: 23

In [19]:
df_ideb = df_ideb.drop([0,1,2], axis='index')
df_ideb = df_ideb.drop([43610, 43611, 43612], axis=0) # notas

df_ideb.columns = [i.upper() for i in ideb_dic_cols.keys()]
df_ideb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43607 entries, 3 to 43609
Data columns (total 84 columns):
SIGLA_UF                         43607 non-null object
COD_MUNICIPIO                    43607 non-null float64
NOME_MUNICIPIO                   43607 non-null object
COD_INEP                         43607 non-null float64
NOME_ESCOLA                      43607 non-null object
REDE                             43607 non-null object
TAXA_APROVACAO_2005_TODOS        43607 non-null object
TAXA_APROVACAO_2005_6            43607 non-null object
TAXA_APROVACAO_2005_7            43607 non-null object
TAXA_APROVACAO_2005_8            43607 non-null object
TAXA_APROVACAO_2005_9            43607 non-null object
TAXA_APROVACAO_2005_INDICADOR    43607 non-null object
TAXA_APROVACAO_2007_TODOS        43607 non-null object
TAXA_APROVACAO_2007_6            43607 non-null object
TAXA_APROVACAO_2007_7            43607 non-null object
TAXA_APROVACAO_2007_8            43607 non-null object
TAXA_APRO

In [20]:
df_ideb.tail(2)

,SIGLA_UF,COD_MUNICIPIO,NOME_MUNICIPIO,COD_INEP,NOME_ESCOLA,REDE,TAXA_APROVACAO_2005_TODOS,TAXA_APROVACAO_2005_6,TAXA_APROVACAO_2005_7,TAXA_APROVACAO_2005_8,...,IDEB_2015,IDEB_2017,PROJECOES_2007,PROJECOES_2009,PROJECOES_2011,PROJECOES_2013,PROJECOES_2015,PROJECOES_2017,PROJECOES_2019,PROJECOES_2021
43608,DF,5300108.0,Brasília,53068149.0,CEF 32 DE CEILANDIA,Estadual,-,-,-,-,...,3.8,4,-,-,-,-,4.1,4.4,4.6,4.9
43609,DF,5300108.0,Brasília,53068211.0,CEF MIGUEL ARCANJO,Estadual,-,-,-,-,...,4.2,-,-,-,-,-,4,4.2,4.5,4.7


In [21]:
df_ideb = df_ideb[df_ideb['REDE'] == 'Estadual']
df_ideb = df_ideb[df_ideb['SIGLA_UF'] == 'RJ']

df_ideb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 967 entries, 25378 to 27683
Data columns (total 84 columns):
SIGLA_UF                         967 non-null object
COD_MUNICIPIO                    967 non-null float64
NOME_MUNICIPIO                   967 non-null object
COD_INEP                         967 non-null float64
NOME_ESCOLA                      967 non-null object
REDE                             967 non-null object
TAXA_APROVACAO_2005_TODOS        967 non-null object
TAXA_APROVACAO_2005_6            967 non-null object
TAXA_APROVACAO_2005_7            967 non-null object
TAXA_APROVACAO_2005_8            967 non-null object
TAXA_APROVACAO_2005_9            967 non-null object
TAXA_APROVACAO_2005_INDICADOR    967 non-null object
TAXA_APROVACAO_2007_TODOS        967 non-null object
TAXA_APROVACAO_2007_6            967 non-null object
TAXA_APROVACAO_2007_7            967 non-null object
TAXA_APROVACAO_2007_8            967 non-null object
TAXA_APROVACAO_2007_9            967 no

In [30]:
df_ideb['SAEB_2015_MEDIA_PADRAO'].value_counts()[:10]

-                   259
4.76833333333333      2
4.33916666666667      2
5.607                 2
5.06066666666667      2
4.51983333333333      2
3.79433333333333      2
4.281                 2
4.64516666666667      2
4.49933333333333      2
Name: SAEB_2015_MEDIA_PADRAO, dtype: int64

In [32]:
df_ideb['TAXA_APROVACAO_2015_9'].value_counts()[:10]

-       252
100      35
92.6     10
83.3      9
75        9
92.3      9
88.5      8
85.5      7
91.3      7
88.2      7
Name: TAXA_APROVACAO_2015_9, dtype: int64

In [31]:
df_ideb['IDEB_2015'].value_counts()[:10]

-      259
4       42
3.6     40
3.8     40
3.7     40
3       34
3.9     32
4.2     31
4.5     31
3.5     30
Name: IDEB_2015, dtype: int64

In [27]:
len(df_ideb[df_ideb['IDEB_2015'] != '-'])

708

In [34]:
df_ideb.to_csv(TREAT_PATH / 'ideb_2015_rj.csv')

## Fogo Cruzado

In [41]:
fogo_2016 = pd.read_csv(RAW_PATH / 'fogo_cruzado_tiroteios.csv', sep=';', encoding='latin-1')
fogo_2016.head()

,id_ocorrencia,local_ocorrencia,latitude_ocorrencia,longitude_ocorrencia,data_ocorrencia,hora_ocorrencia,presen_agen_segur_ocorrencia,qtd_morto_civil_ocorrencia,qtd_morto_agen_segur_ocorrencia,qtd_ferido_civil_ocorrencia,...,cidade_id,nome_cidade,cod_ibge_cidade,gentilico_cidade,populacao_cidade,area_cidade,densidade_demo_cidade,nome_estado,uf_estado,cod_ibge_estado
0,16,"Santa Cruz, Rio de Janeiro - RJ, Brasil",-22.919651,-43.684387,05/07/2016,6:02:00,1,0,1,0,...,3661,Rio de Janeiro,3304557,carioca,6320446,119746,5.27,Rio de Janeiro,RJ,33
1,17,"Rua Barão - Praça Seca, Rio de Janeiro - RJ, B...",-22.898642,-43.352360,05/07/2016,13:44:00,1,0,0,0,...,3661,Rio de Janeiro,3304557,carioca,6320446,119746,5.27,Rio de Janeiro,RJ,33
2,18,"Estrada da Covanca - Tanque, Rio de Janeiro - ...",-22.914858,-43.348198,05/07/2016,13:48:00,1,0,0,0,...,3661,Rio de Janeiro,3304557,carioca,6320446,119746,5.27,Rio de Janeiro,RJ,33
3,21,"Complexo do Alemão, Rio de Janeiro - RJ, Brasil",-22.859333,-43.275475,05/07/2016,16:32:00,0,0,0,0,...,3661,Rio de Janeiro,3304557,carioca,6320446,119746,5.27,Rio de Janeiro,RJ,33
4,22,"Santa Cruz, Rio de Janeiro - RJ, Brasil",-22.919651,-43.684387,05/07/2016,16:35:00,0,0,0,0,...,3661,Rio de Janeiro,3304557,carioca,6320446,119746,5.27,Rio de Janeiro,RJ,33


In [46]:
import geopandas as gpd
from shapely.geometry import Point

fogo_2016 = gpd.GeoDataFrame(fogo_2016, geometry=[Point(x, y) for x, y in zip(fogo_2016['longitude_ocorrencia'], fogo_2016['latitude_ocorrencia'])])
fogo_2016.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2204 entries, 0 to 2203
Data columns (total 24 columns):
id_ocorrencia                       2204 non-null int64
local_ocorrencia                    2204 non-null object
latitude_ocorrencia                 2204 non-null float64
longitude_ocorrencia                2204 non-null float64
data_ocorrencia                     2204 non-null object
hora_ocorrencia                     2204 non-null object
presen_agen_segur_ocorrencia        2204 non-null int64
qtd_morto_civil_ocorrencia          2204 non-null int64
qtd_morto_agen_segur_ocorrencia     2204 non-null int64
qtd_ferido_civil_ocorrencia         2204 non-null int64
qtd_ferido_agen_segur_ocorrencia    2204 non-null int64
fonte_ocorrencia                    2204 non-null int64
estado_id                           2204 non-null int64
cidade_id                           2204 non-null int64
nome_cidade                         2204 non-null object
cod_ibge_cidade                     2

In [50]:
fogo_2016.to_file(TREAT_PATH / 'fogo_2016_rj.shp')